# Deploy the Model

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

![Pipeline](img/generative_ai_pipeline_rlhf_plus.png)

In [10]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=32877658112, available=5125627904, percent=84.4, used=27279261696, free=429350912, active=30810112000, inactive=391979008, buffers=0, cached=5169045504, shared=1896448, slab=498511872)


In [11]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

# Retrieve model endpoint


In [12]:
%store -r pipeline_endpoint_name

In [13]:
try:
    pipeline_endpoint_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run previous notebooks before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [14]:
print(pipeline_endpoint_name)

model-from-registry-ep-1682528395


In [15]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker HTTPS Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_
_Note:  This will take a few minutes.  Please be patient._

In [16]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

CPU times: user 15.3 ms, sys: 8.02 ms, total: 23.4 ms
Wall time: 192 ms


# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# Zero Shot Inference

In [30]:
import json
from sagemaker import Predictor
from sagemaker import serializers, deserializers

prompt = """Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary:"""
predictor = Predictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),        # will convert python dict to json
    deserializer=deserializers.JSONDeserializer(),  # will convert json to python dict    
)
prompt_json = json.dumps({"inputs": prompt})
response = predictor.predict(prompt_json,
        {
#            "ContentType": "application/x-text",
            "ContentType": "application/json",            
            "Accept": "application/json",
        },
)
response_json = json.loads(response.decode('utf-8'))
print(response_json)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (424) from primary with message "{
  "code":424,
  "message":"prediction failure",
  "error":"string indices must be integers"
}". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/model-from-registry-ep-1682528395 in account 079002598131 for more information.

# Test predictions

In [24]:
from random import randint
import pandas as pd

In [47]:
# Load the train and test data into a pandas dataframe
#train_data = pd.read_json("data/dialogsum.train.jsonl", lines=True)
test_data = pd.read_json("data/dialogsum.test.jsonl", lines=True)
#print ("Train data shape: ", train_data.shape)
print ("Test data shape: ", test_data.shape)

Test data shape:  (500, 8)


In [48]:
# !wget https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.train.jsonl -O data/dialogsum.train.jsonl
# !wget https://raw.githubusercontent.com/cylnlp/dialogsum/main/DialogSum_Data/dialogsum.test.jsonl -O data/dialogsum.test.jsonl
# # Load the train and test data into a pandas dataframe
# train_data = pd.read_json("data/dialogsum.train.jsonl", lines=True)
# test_data = pd.read_json("data/dialogsum.test.jsonl", lines=True)
# print ("Train data shape: ", train_data.shape)
# print ("Test data shape: ", test_data.shape)

In [49]:
random_dialogue_idx = randint(0, test_data.shape[0])
random_dialogue = test_data["dialogue"][random_dialogue_idx]

output = predictor.predict({"inputs": [random_dialogue], "parameters":{"max_length": 100}})
output_summary = output["outputs"]

print("#####DIALOGUE######\n", random_dialogue)
print("\n#####GENERATED SUMMARY######\n", output_summary)
print("\n#####HUMAN BASELINE 1######\n", test_data["summary1"][random_dialogue_idx])
print("\n#####HUMAN BASELINE 2######\n", test_data["summary2"][random_dialogue_idx])
print("\n#####HUMAN BASELINE 3######\n", test_data["summary3"][random_dialogue_idx])

#####DIALOGUE######
 #Person1#: OK, Mrs Thomas. I finished cleaning up the leaves in the yard.
#Person2#: Oh, thank you, James.
#Person1#: I also swept the front steps and took out the garbage. Is there anything else I can help you with?
#Person2#: Can you also take my dog Oscar for a walk?
#Person1#: Sure.
#Person2#: Here is such a prince. Ever since I broke my leg i t has been so difficult to take care of things around the house. My daughter Leah helps out on Mondays and Tuesdays and my son Robbie on Thursdays, but I'm so happy you can come on Saturdays.
#Person1#: Well, I appreciate the weekend work. I'm saving up to buy a bike so I can ride to my basketball practice.
#Person2#: Will you still be able to work for me after you get your bike?
#Person1#: Of course, but don't forget I'm away next weekend visiting my grandfather. My sister Allison will come by instead.
#Person2#: OK.

#####GENERATED SUMMARY######
 I'm sorry, but I'm sorry, but I'm sorry, but I'm sorry, but I'm sorry, but

# Clean Up: Tear Down Endpoint

In [ ]:
sm.delete_endpoint(
    EndpointName=pipeline_endpoint_name
)